In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [7]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [43]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36980, 674)
(36980,)
(20984, 674)
(20984,)


In [45]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,5,0,0,0,0,0,7,0,0,0,...,0,0,18,0,0,0,0,0,2023-01-29 00:00:00,HB101
1,24,0,0,0,0,0,7,0,0,0,...,0,0,3,0,0,0,0,0,2023-01-29 12:00:00,HB101
2,2,0,0,0,0,0,13,0,0,0,...,0,0,13,0,0,0,0,0,2023-01-30 00:00:00,HB101
3,34,0,0,0,0,0,18,0,0,0,...,0,0,17,0,0,0,0,0,2023-01-30 12:00:00,HB101
4,1,0,0,0,0,0,11,0,0,0,...,0,0,24,0,0,0,0,0,2023-01-31 00:00:00,HB101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36975,36,0,0,0,0,0,24,0,0,0,...,0,0,1,0,0,0,0,0,2023-08-29 12:00:00,JC116
36976,1,0,0,0,0,0,10,0,0,0,...,0,0,27,0,0,0,0,0,2023-08-30 00:00:00,JC116
36977,32,0,0,0,0,0,22,0,0,0,...,0,0,5,0,0,0,0,0,2023-08-30 12:00:00,JC116
36978,3,0,0,0,0,0,13,0,0,0,...,0,0,24,0,0,0,0,0,2023-08-31 00:00:00,JC116


In [48]:
from src.pipeline_utils import get_pipeline, format_predictions
pipeline = get_pipeline()

In [49]:
pipeline

Pipeline(steps=[('numericfeatureselector', NumericFeatureSelector()),
                ('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x16d3fb060>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [52]:
# Get your pipeline
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

# Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10690
[LightGBM] [Info] Number of data points in the train set: 36980, number of used features: 115
[LightGBM] [Info] Start training from score 6.905057


In [54]:
y_pred

array([ 0.69948779, 44.2803079 ,  5.29287255, ..., 15.19298491,
        2.03246091, 11.08187414])

In [55]:
from sklearn.metrics import mean_absolute_error

In [56]:
test_mae = mean_absolute_error(y_test, y_pred)
print(f"{test_mae:.4f}")

2.6619


In [57]:

import src.config as config

In [60]:
import joblib  


# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['/Users/vamsisaigarapati/Documents/github/citibike_ride_prediction/models/lgb_model.pkl']

In [24]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

2025-05-09 22:10:46,982 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 22:10:46,989 INFO: Initializing external client
2025-05-09 22:10:46,989 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 22:10:47,606 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637


In [30]:
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213637/fs/1201260/fv/time_series_hourly_feature_view_citi_bike/version/1
Feature view 'time_series_hourly_feature_view_citi_bike' (version 1) created successfully.
Feature view 'time_series_hourly_feature_view_citi_bike' (version 1) retrieved successfully.


In [74]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [75]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name=config.MODEL_NAME,
    metrics={"test_mae": test_mae},
    description="LightGBM model for bike rides",
    input_example=X_train.sample(),
    model_schema=model_schema,
    #include_files=["src/", "requirements.txt"]  # Include directories and files
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
model.save(str(config.MODELS_DIR) + "/lgb_model.pkl")
# model.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/vamsisaigarapati/Documents/github/citibike_ride_prediction/models/lgb_model.pkl: 0.000%|     …

Uploading /Users/vamsisaigarapati/Documents/github/citibike_ride_prediction/notebooks/input_example.json: 0.00…

Uploading /Users/vamsisaigarapati/Documents/github/citibike_ride_prediction/notebooks/model_schema.json: 0.000…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1213637/models/model_demand_predictor_next6hours/3


Model(name: 'model_demand_predictor_next6hours', version: 3)

In [62]:
model

Model(name: 'model_demand_predictor_next6hours', version: 2)

In [63]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [64]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-09 23:43:49,375 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-09 23:43:49,379 INFO: Initializing external client
2025-05-09 23:43:49,379 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 23:43:49,848 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637


Downloading: 0.000%|          | 0/313579 elapsed<00:00 remaining<?

In [65]:
model

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x17f55b920>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [68]:
from datetime import datetime, timedelta

import pandas as pd
import pytz
import src.config as config
from src.data_utils import transform_ts_data_info_features_and_target
from src.inference import (
    get_feature_store
)

# Get the current datetime64[us, Etc/UTC]
# for number in range(22, 24 * 29):
# current_date = pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=number)
est = pytz.timezone("America/New_York")

# Get current timestamp in EST
current_date = pd.Timestamp.now(tz=est)
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1 * 29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

2025-05-09 23:48:36,763 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 23:48:36,767 INFO: Initializing external client
2025-05-09 23:48:36,767 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 23:48:37,411 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Fetching data from 2025-04-10 23:48:36.762871-04:00 to 2025-05-09 22:48:36.762871-04:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 


In [69]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-05-03 20:00:00,JC108,0
1,2025-04-14 07:00:00,JC009,0
2,2025-04-19 05:00:00,HB301,0
3,2025-05-02 08:00:00,JC075,0
4,2025-05-02 12:00:00,JC053,2
...,...,...,...
55854,2025-04-21 06:00:00,HB105,13
55855,2025-05-09 12:00:00,HB501,11
55856,2025-05-07 19:00:00,JC052,0
55857,2025-05-04 09:00:00,JC105,0


In [70]:
from src.inference import (
    get_feature_store,
    get_model_predictions,
    load_model_from_registry,
)

In [72]:
from src.data_utils import transform_ts_data_info_features

features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=12)

model = load_model_from_registry()

predictions = get_model_predictions(model, features)
predictions["pickup_hour"] = current_date.ceil("h")
print(predictions)

2025-05-09 23:51:10,195 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 23:51:10,199 INFO: Initializing external client
2025-05-09 23:51:10,199 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 23:51:10,902 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637


Downloading: 0.000%|          | 0/313579 elapsed<00:00 remaining<?

   pickup_location_id  predicted_demand               pickup_hour
0               JC108               0.0 2025-05-10 00:00:00-04:00
1               JC009               6.0 2025-05-10 00:00:00-04:00
2               HB301               0.0 2025-05-10 00:00:00-04:00
3               JC075               0.0 2025-05-10 00:00:00-04:00
4               JC053               1.0 2025-05-10 00:00:00-04:00
..                ...               ...                       ...
78              JC098               6.0 2025-05-10 00:00:00-04:00
79              HB305               6.0 2025-05-10 00:00:00-04:00
80              JC034               0.0 2025-05-10 00:00:00-04:00
81              JC105               0.0 2025-05-10 00:00:00-04:00
82              HB503               0.0 2025-05-10 00:00:00-04:00

[83 rows x 3 columns]


In [73]:


feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

feature_group.insert(predictions, write_options={"wait_for_job": False})


2025-05-09 23:51:24,250 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 23:51:24,254 INFO: Initializing external client
2025-05-09 23:51:24,254 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 23:51:24,860 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213637/fs/1201260/fg/1454547


Uploading Dataframe: 100.00% |██████████| Rows 83/83 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: bike_6hours_model_prediction_citibike_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213637/jobs/named/bike_6hours_model_prediction_citibike_1_offline_fg_materialization/executions


(Job('bike_6hours_model_prediction_citibike_1_offline_fg_materialization', 'SPARK'),
 None)